In [2]:
import time
import os
import csv
import torch
import numpy as np
import math
import itertools
from itertools import product
import torch.nn.functional as F
import pandas as pd
from scipy.stats import poisson
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA AVAILABLE" if device=='cuda' else "cuda NOT available")

cuda NOT available


In [2]:
def soft_shift_poisson(rate:int, left_shift:int=0) -> float:
    """
    Returns the mathematical expectation of max(0,X-left_shift), where X a Poisson distribution.
    """
    if left_shift == 0: return rate
    sub_probability = poisson.cdf(2*left_shift-1,rate)
    sub_mean =  0
    for n in range(2*left_shift): sub_mean += n*poisson.pmf(n,rate)
    remaining_mean = rate - sub_mean
    remaining_probability = 1 - sub_probability
    return remaining_mean - remaining_probability*left_shift

In [3]:
class ground_model:
    """
    Definition and operations for a radio resource allocation problem.
    Parameters:
    max_size = maximum number of bits in an user equipment's queue;
    number of user equipments; number of resource blocks;
    vector_arrival_rates = vector of arrival rate for each user equipment's buffer;
    CQIs_are_equal: indicates if the chanel quality index is constant in the UE and in the RB;
    CQI_base = number of bits scheduled for transmission when CQI=1;
    coef_of_drop, coef_of_latency, power_of_drop, power_of_latency = α, β, x, y:
    cost = α*(cost for rejection)**x + β(cost for latency)**y;
    discount factor;
    precision = bound of the difference between the calculated and the optimal value function of the MDP;
    path = location of the results of the operations (if it is not indicated from the root, the location is in the current directory).
    """
    
    def __init__(self, max_size:int=2, number_UEs:int=2, number_RBs:int=2, 
                 vector_arrival_rates=None,
                 CQIs_are_equal=True, CQI_base:float=1,
                 coef_of_drop:float=1, coef_of_latency:float=1, power_of_drop:float=1, power_of_latency:float=1, 
                 discount_factor:float=0.9, precision:float=1e-6,
                 path=None) -> None:
        
        # States and actions
        number_states, number_actions = (max_size + 1) * number_UEs, number_UEs ** number_RBs
        range_UE_indices = torch.tensor(range(number_UEs))
        range_number_bits = torch.tensor(range(max_size+1))
        list_states = list(product(range_number_bits,repeat=number_UEs))
        list_actions = list(product(range_UE_indices, repeat=number_RBs))
        
        # Send to the device: the data, the ranges for indices, the states and the actions
        self.max_size = torch.tensor([max_size],dtype=int).to(device)
        self.number_UEs, self.number_RBs = torch.tensor([number_UEs],dtype=int).to(device), torch.tensor([number_RBs],dtype=int).to(device)
        self.coef_of_drop, self.coef_of_latency = torch.tensor([coef_of_drop],dtype=float).to(device), torch.tensor([coef_of_latency],dtype=float).to(device)
        self.power_of_drop, self.power_of_latency = torch.tensor([power_of_drop],dtype=float).to(device), torch.tensor([power_of_latency],dtype=float).to(device)
        self.discount_factor, self.precision = torch.tensor([discount_factor], dtype=float).to(device), torch.tensor([precision], dtype=float).to(device)
        self.number_states, self.number_actions = torch.tensor([number_states]).to(device), torch.tensor([number_actions]).to(device)
        self.range_UE_indices, self.range_RB_indices = torch.tensor(range(number_UEs)).to(device), torch.tensor(range(number_RBs)).to(device)
        self.range_state_indices, self.range_action_indices = torch.tensor(range(number_states)).to(device), torch.tensor(range(number_actions)).to(device)
        self.matrix_states, self.matrix_actions = torch.tensor(list_states).to(device), torch.tensor(list_actions).to(device)
        self.empty_buffer = torch.zeros([self.number_UEs],dtype=int).to(device)                      # Number of bits in each UE's empty buffer
        self.full_buffer = self.max_size*torch.ones([self.number_UEs],dtype=int).to(device)          # Number of bits in each UE's full buffer
        
        # Get the arrival rates
        if vector_arrival_rates==None:
            vector_arrival_rates = torch.ones([self.number_UEs]).to(device)
            print("Setting the arrival rate for each UE.")
            decision = input("Same arrival rate? (y/n)")
            if "Y" in decision.upper():
                value_ = float(input("For all UE, arrival rate is: "))
                value_ = torch.tensor([value_],dtype=float).to(device)
                vector_arrival_rates = value_*vector_arrival_rates
            else:
                for i in self.range_UE_indices:
                    print(f"Arrival rate for UE {i}: ")
                    vector_arrival_rates[i] = float(input())
        else:
            vector_arrival_rates = torch.ones([self.number_UEs]).to(device)
            fraction = torch.tensor([max_size/3],dtype=float).to(device)
            vector_arrival_rates = fraction*vector_arrival_rates
        self.vector_arrival_rates = vector_arrival_rates
        
        # Get the chanel quality indices matrix
        if CQIs_are_equal:
            self.CQI_matrix = CQI_base*torch.ones([self.number_UEs,number_RBs],dtype=float).to(device)
        else:
            self.CQI_matrix = torch.ones([self.number_UEs,number_RBs],dtype=float)
            print("Definition of the chanel quality indices.")
            decision = input("Allow various values of CQI? (y/n): ")
            if "Y" in decision.upper():
                print("\nFor each UE and each RB, let's define the CQI as:  CQI = (coef of UE and RB)*(common base)")
                print("Enter all the coefficients. Later on, you will enter the common base to multiply.")
                for i,j in product(self.range_UE_indices,self.range_RB_indices):
                    print(f"    Coefficient for UE {i} and RB {j}: ", end="")
                    value_ = input()
                    if value_.isnumeric(): 
                        value_ = torch.tensor([value_],dtype=int).to(device)
                        self.CQI_matrix[i,j] = value_
                value_ = input("Enter the common base: ")
                if value_.isnumeric(): 
                    value_ = torch.tensor([value_],dtype=int).to(device)
                    CQI_base = value_
                self.CQI_matrix = CQI_base*self.CQI_matrix
                self.CQI_matrix = self.CQI_matrix.to(device)
    
    def remainder_fn(self,state, action) -> torch.tensor:
        """
        Returns the number of bits remaining in each UE's buffer if action of index action_index was taken in state of index state_index.
        """
        schedule = torch.zeros([self.number_UEs],dtype=float).to(device)  # Number of bits scheduled for transmission
        for i,j in product(self.range_UE_indices,self.range_RB_indices): 
            if action[j]==i:
                schedule[i] += self.CQI_matrix[i,j]
        schedule = schedule.to(int)
        angry_remainder = state - schedule    # Number of remaining bits if negative values are possible
        return torch.max(self.empty_buffer, angry_remainder)
    
    def transition_probability_fn(self, state1:int, action:int, state2:int) -> torch.tensor:
        def UE_transition_probability_fn(rest:int, rate:float, target:int) -> torch.tensor:
            max_size = self.max_size
            sure, impossible = torch.tensor([1],dtype=float).to(device), torch.tensor([0],dtype=float).to(device)
            if target<rest:     return impossible                                                          # target < rest
            if target<max_size: return torch.tensor([poisson.pmf(target-rest,rate)],dtype=float).to(device) # rest ≤ target < max_size  
            range_targets = torch.tensor(range(rest.item(),max_size.item()),dtype=int).to(device)
            probability = sure                                                           # target = max_size:  need to exploit the others
            for t in range_targets: probability = probability - UE_transition_probability_fn(rest,rate,t)
            return probability
        rate = self.vector_arrival_rates
        remainder = self.remainder_fn(state1,action)
        probability = torch.tensor([1],dtype=float).to(device)
        for i in self.range_UE_indices: 
            probability = probability * UE_transition_probability_fn(remainder[i], rate[i], state2[i])
        return probability
    
    def cost_fn(first_queue_index:int, action_index:int) -> float:
        9

In [4]:
model = ground_model()

Setting the arrival rate for each UE.


In [5]:
ai = torch.randint(0, model.number_actions, (1,)).item()
si1 = torch.randint(0, model.number_states, (1,)).item()
si2 = torch.randint(0, model.number_states, (1,)).item()
action, state1, state2 = model.matrix_actions[ai], model.matrix_states[si1], model.matrix_states[si2]
Naction, Nstate1, Nstate2 = action.numpy(), state1.numpy(), state2.numpy()
print(f"Getting the transition probability under   a = {Naction}  from   s = {Nstate1}   to    s = {Nstate2} ...\n")
M = model.transition_probability_fn(state1, action, state2).item()
print(f"\n\n\nResult = {M}")

Getting the transition probability under   a = [1 0]  from   s = [1 0]   to    s = [1 0] ...




Result = 0.16151721439572433


In [6]:
max_diff = 0
for ai in range(model.number_actions):
    AT = model.matrix_actions[ai]
    AN = AT.numpy()
    print(f"\n\nWith action a_{ai} = {AN}\n")
    for si1 in range(model.number_states):
        S1T = model.matrix_states[si1]
        S1N = S1T.numpy()
        S = torch.tensor([0]).to(device)
        for si2 in range(model.number_states):
            S2T = model.matrix_states[si2]
            PT = model.transition_probability_fn(S1T,AT,S2T)
            S = S + PT
            S2N, PN = S2T.numpy(), PT.item()
            print(f"a_{ai}:      {S1N} --->  {S2N}:    P = {PN}")
        SN = S.item()
        print(f"\n                  Sum of probabilities = {SN}\n\n")
        max_diff = max((max_diff, abs(SN-1)))
print(f"\nMax error = {max_diff}")



With action a_0 = [0 0]

a_0:      [0 0] --->  [0 0]:    P = 0.20189651799465538
a_0:      [0 0] --->  [0 1]:    P = 0.16151721439572433
a_0:      [0 0] --->  [0 2]:    P = 0.08591523172684185
a_0:      [0 0] --->  [1 0]:    P = 0.16151721439572433
a_0:      [0 0] --->  [1 1]:    P = 0.12921377151657948
a_0:      [0 0] --->  [1 2]:    P = 0.06873218538147349

                  Sum of probabilities = 0.8087921354109988


a_0:      [0 1] --->  [0 0]:    P = 0.0
a_0:      [0 1] --->  [0 1]:    P = 0.20189651799465538
a_0:      [0 1] --->  [0 2]:    P = 0.24743244612256618
a_0:      [0 1] --->  [1 0]:    P = 0.0
a_0:      [0 1] --->  [1 1]:    P = 0.16151721439572433
a_0:      [0 1] --->  [1 2]:    P = 0.19794595689805297

                  Sum of probabilities = 0.8087921354109988


a_0:      [0 2] --->  [0 0]:    P = 0.0
a_0:      [0 2] --->  [0 1]:    P = 0.0
a_0:      [0 2] --->  [0 2]:    P = 0.44932896411722156
a_0:      [0 2] --->  [1 0]:    P = 0.0
a_0:      [0 2] --->  [1 1]:   

In [161]:
b = [a for a in range(5)]
print(b)

[0, 1, 2, 3, 4]
